# Global Impact of Brexit Uncertainty

In this notebook we will explore the global impact of Brexit uncertainty using a text-based method for measuring the cross-border propagation of large shocks at the firm level, as described by Hassan et al. ([2024](https://doi.org/10.1111/jofi.13293))


```math
BrexitExposure_{i,t} = \frac{1}{B_{i,t}}\sum_{b=1}^{B_{i,t}} 1\left[b=Brexit\right]
```

(Hassan et al., 2024, p. 419)


```math
BrexitRisk_{i,t} = \frac{1}{B_{i,t}}\sum_{b=1}^{B_{i,t}} \left\{1\left[b=Brexit\right] \times 1\left[|b-r| \lt 10\right]\right\}
```

(Hassan et al., 2024, p. 420)


```math
BrexitSentiment_{i,t} = \frac{1}{B_{i,t}}\sum_{b=1}^{B_{i,t}} \left\{1\left[b=Brexit\right] \times \left(\sum_{c=b-10}^{b+10}{S(c)}\right)\right\}
```

(Hassan et al., 2024, p. 420)


```math
NonBrexitRisk_{i,t} = \frac{1}{B_{i,t}}\sum_{b}^{B_{i,t}} \left\{1\left[b\in\R\right]\right\} - BrexitRisk_{i,t}
```

(Hassan et al., 2024, p. 421)


```math
NonBrexitSentiment_{i,t} = \frac{1}{B_{i,t}}\sum_{b}^{B_{i,t}} S(b) - BrexitSentiment_{i,t}
```

(Hassan et al., 2024, p. 421)


In [58]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from process_transcripts import process_transcripts
from master_dictionary import load_masterdictionary
from pathlib import Path
from transcript_stats import get_transcript_stats

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Process all the transcripts if we haven't already.


In [59]:
process_transcripts()

Processing transcripts/2013/2013-Nov-12-DPWGn.DE-140654996696-Transcript.txt...
  Skipping 2013-Nov-12-DPWGn.DE-140654996696-Transcript.txt, already processed.
Processing transcripts/2013/2013-Apr-17-TSCO.L-138043584744-Transcript.txt...
  Skipping 2013-Apr-17-TSCO.L-138043584744-Transcript.txt, already processed.
Processing transcripts/2013/2013-Jan-29-LLY.N-138602223739-Transcript.txt...
  Skipping 2013-Jan-29-LLY.N-138602223739-Transcript.txt, already processed.
Processing transcripts/2013/2013-Mar-05-DPWGn.DE-138356575985-Transcript.txt...
  Skipping 2013-Mar-05-DPWGn.DE-138356575985-Transcript.txt, already processed.
Processing transcripts/2013/2013-Dec-05-KR.N-140861526577-Transcript.txt...
  Skipping 2013-Dec-05-KR.N-140861526577-Transcript.txt, already processed.
Processing transcripts/2013/2013-Jul-16-JNJ.N-140981724350-Transcript.txt...
  Skipping 2013-Jul-16-JNJ.N-140981724350-Transcript.txt, already processed.
Processing transcripts/2013/2013-Jan-29-F.N-140697301708-Transcr

## Get Brexit Metrics


In [60]:
df = get_transcript_stats()

## Data Looks Good


In [61]:
df

,ticker,year,month,day,filename,BrexitExposure,BrexitRisk,BrexitSentiment,NonBrexitRisk,NonBrexitSentiment
0,DPWGn.DE,2013,11,12,/Users/mfryer/Code/brexit-uncertainty/src/proc...,0.000000,0.0,-0.000000,4.0,73.000000
1,LLY.N,2013,04,24,/Users/mfryer/Code/brexit-uncertainty/src/proc...,0.000000,0.0,-0.000000,1.0,2.000000
2,F.N,2013,10,24,/Users/mfryer/Code/brexit-uncertainty/src/proc...,0.000000,0.0,-0.000000,4.0,9.000000
3,AZN.L,2013,10,31,/Users/mfryer/Code/brexit-uncertainty/src/proc...,0.000000,0.0,-0.000000,10.0,32.000000
4,BARC.L,2013,02,13,/Users/mfryer/Code/brexit-uncertainty/src/proc...,0.000000,0.0,-0.000000,14.0,3.000000
...,...,...,...,...,...,...,...,...,...,...
336,KR.N,2016,03,03,/Users/mfryer/Code/brexit-uncertainty/src/proc...,0.000000,0.0,-0.000000,4.0,77.000000
337,F.N,2016,04,28,/Users/mfryer/Code/brexit-uncertainty/src/proc...,0.000225,0.0,24.444444,9.0,113.555556
338,F.N,2016,10,27,/Users/mfryer/Code/brexit-uncertainty/src/proc...,0.001253,0.0,-0.000000,5.0,67.000000
339,7203.T,2016,05,11,/Users/mfryer/Code/brexit-uncertainty/src/proc...,0.000000,0.0,-0.000000,0.0,0.000000
